In [9]:
# Import essential packages
import wntr
import pickle
import networkx as nx
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
# "the default sans-serif font is Arial"
matplotlib.rcParams['font.sans-serif'] = "Arial"
# Then, "ALWAYS use sans-serif fonts"
matplotlib.rcParams['font.family'] = "sans-serif"
matplotlib.rcParams.update({'font.size': 8})

In [15]:
# Load the backbone network
FBackbone_network = open(r'CoH_Backbone_network.pickle', 'rb')
Backbone_network = pickle.load(FBackbone_network)
FBackbone_network.close()

In [11]:
Wdn = wntr.network.WaterNetworkModel(r'S_CoH_460MGD_2018.inp')

C:\Users\Zhouxiaomu\anaconda3\lib\site-packages\wntr\epanet\io.py:2085: UserWarning: Not all curves were used in "S_CoH_460MGD_2018.inp"; added with type None, units conversion left to user
  warnings.warn('Not all curves were used in "{}"; added with type None, units conversion left to user'.format(self.wn.name))


In [12]:
# Remove node and links from Wdn
def remove_elements(wdn, backbone_network):
    # Remove the links that either one or two end nodes not in the dgraph
    backbone_nodes = list(backbone_network.nodes)
    link_remove = []
    node_remove = []
    for name, link in wdn.links():
        start_node = link.start_node_name
        end_node = link.end_node_name
        if (start_node not in backbone_nodes) and (end_node not in backbone_nodes):
            link_remove.append(name)
            node_remove.append(start_node)
            node_remove.append(end_node)
        elif start_node not in backbone_nodes:
            link_remove.append(name)
            node_remove.append(start_node)
        elif end_node not in backbone_nodes:
            link_remove.append(name)
            node_remove.append(end_node)
    for lr in link_remove:
        wdn.remove_link(lr, force=True)
    for nr in node_remove:
        try:
            wdn.remove_node(nr, force=True)
        except:
            pass    
    return wdn

In [16]:
Wdn_modified = remove_elements(Wdn, Backbone_network)

In [17]:
Wdn_modified.write_inpfile(r'CoH_Backbone_Final.inp')